# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [143]:
# import libraries
import re
import nltk
from sqlalchemy import create_engine
import pandas as pd
import numpy as np


from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


import time

[nltk_data] Downloading package punkt to /Users/mila/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/mila/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mila/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [146]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

In [149]:
def load_data(database_filepath):
    '''
    Loads dataset from sqlite database database_filepath and splits data into features X and targets y.
    Args:
        database_filepath - a path to database to load the data
    Returns:
        X - array of features (text messages)
        y - array of targets (0 or 1)
        categories - list of target names
    '''
    # import data from sqlite database and save to df dataframe.
    from sqlalchemy import create_engine
    engine = create_engine('sqlite:///{}'.format(database_filepath))
    df = pd.read_sql_table(database_filepath.split('/')[-1], con=engine)
    
    # split data into features X and target y
    X = df['message'].values
    y = df[df.columns[4:]].values
    
    # create a list of target variables
    categories = list(df.columns[4:])
    
    return X, y, categories

### 2. Write a tokenization function to process your text data

In [150]:
def tokenize(text):
    '''
    Splits a text input into a list of filtered, stemmed and lemmatized tokens. 
    
    Args:
        text - input string
    Returns:
        tokens - a list of tokens
    '''    
    import re

    from nltk.tokenize import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk.stem.porter import PorterStemmer
    # find URLs and replaces them with "urlplaceholder"
    url_regex = '[](?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-&?=%.]+' 
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # remove punctuation and converts to lowercase    
    text = re.sub(r"[^a-zA-z0-9]", " ", text.lower())
    # remove brackets
    text = re.sub(r"[\([{})\]]", "", text)
    # split a text into a list of tokens 
    tokens = word_tokenize(text)
    # removes digits and stopwords
    tokens = [word for word in tokens if word not in stopwords.words("english") and not word.isdigit()]
    # stemming - reducing words to their root form
    tokens = [PorterStemmer().stem(word) for word in tokens]
    # lemmatization - reducing words to their dictionary form
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens]
    
    return tokens

In [196]:
#check tokenize function for the first 10 rows
X, y, category = load_data('../data/DisasterResponse.db')
for message in X[:10]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurrican'] 

Looking for someone but no name
['look', 'someon', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight'] 

Information about the National Palace-
['inform', 'nation', 'palac'] 

Storm at sacred heart of jesus
['storm', 'sacr', 'heart', 'jesu'] 

Please, we need tents and water. We are in Silo, Thank you!
['plea', 'need', 'tent', 'water', 'silo', 'thank'] 

I would like to receive the messages, thank you
['would', 'like', 'receiv', 'messag', 'thank'] 

I am in Croix-des-Bouquets. We have health issues. 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [77]:
def model_pipeline(classifier):
    '''
    Builds a machine learning pipeline:
        1. CountVectorizer() - converts a collection of text documents into a matrix of token counts using tokenize() function 
        2. TfidfTransformer() - transforms a matrix of token countsinto a matrix of TF-IDF scores
        3. MultiOutputClassifier() - multi-output classification using input classifier
    Args:
        classifier - a machine learning classifier
    Returns:
        pipeline - a classification result
    '''  
    pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('clf', MultiOutputClassifier(classifier))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [152]:
#load data from database and split into features and targets variables
X, y, category = load_data('../data/DisasterResponse.db')

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [153]:
#build a machine learning model with RandomForestClassifier 
model = model_pipeline(RandomForestClassifier())

start = time.time()
#fit the model with training data 
model.fit(X_train, y_train)
print('Duration : {} seconds'.format(time.time()-start))

# predict classifiers on test data
y_pred = model.predict(X_test)

Duration : 534.1918458938599 seconds


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [160]:
from sklearn.metrics import accuracy_score

In [161]:
#accuracy score
accuracy_score(y_test, y_pred)

0.270057215511761

In [166]:
#check the training accuracy
print(model.score(X_train, y_train))

0.9934604904632153


In [ ]:
#check the test accuracy
print(model.score(X_test, y_test))

In [171]:
print(confusion_matrix(y_test[:,0],y_pred[:,0]))

[[ 758 1095]
 [ 310 5702]]


In [173]:
print(confusion_matrix(y_test[:,15],y_pred[:,15]))

[[7584    4]
 [ 270    7]]


In [168]:
print(classification_report(y_test[:,0],y_pred[:,0]))

              precision    recall  f1-score   support

           0       0.71      0.41      0.52      1853
           1       0.84      0.95      0.89      6012

    accuracy                           0.82      7865
   macro avg       0.77      0.68      0.70      7865
weighted avg       0.81      0.82      0.80      7865



In [172]:
print(classification_report(y_test[:,15],y_pred[:,15]))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      7588
           1       0.64      0.03      0.05       277

    accuracy                           0.97      7865
   macro avg       0.80      0.51      0.52      7865
weighted avg       0.95      0.97      0.95      7865



In [157]:
print(classification_report(y_test, y_pred, target_names=category, zero_division=0))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      6012
               request       0.84      0.50      0.63      1313
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.68      0.72      3255
          medical_help       0.64      0.07      0.13       629
      medical_products       0.83      0.05      0.09       406
     search_and_rescue       0.80      0.10      0.18       202
              security       0.00      0.00      0.00       151
              military       0.67      0.07      0.12       244
                 water       0.89      0.36      0.51       493
                  food       0.85      0.60      0.70       861
               shelter       0.82      0.32      0.46       715
              clothing       0.67      0.07      0.13       114
                 money       0.89      0.05      0.09       174
        missing_people       1.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [108]:
model_pipeline(RandomForestClassifier()).get_params()

{'memory': None,
 'steps': [('text_pipeline', Pipeline(steps=[('vect',
                    CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>)),
                   ('tfidf', TfidfTransformer())])),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'text_pipeline': Pipeline(steps=[('vect',
                  CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>)),
                 ('tfidf', TfidfTransformer())]),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'text_pipeline__memory': None,
 'text_pipeline__steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>)),
  ('tfidf', TfidfTransformer())],
 'text_pipeline__verbose': False,
 'text_pipeline__vect': CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>),
 'text_pipeline__tfidf': TfidfTransformer(),
 'text_pipeline__vect__analyzer': 'word',
 'text_pipeline__vect__binary': False,
 'text_pipeline__vect__decode_err

In [58]:
#test different parameters with GridSearchCV
parameters = {
    'text_pipeline__vect__max_df': [.85, 1]
   ,'clf__estimator__n_estimators': [100, 120, 140]
   ,'clf__estimator__max_features': ['sqrt', 'log2', None]
   ,'clf__estimator__max_depth': [3, 5, 7, None]
}

start = time.time()
cv = GridSearchCV(model_pipeline(RandomForestClassifier()), parameters, n_jobs=4, cv=3)
    
cv.fit(X_train, y_train)
print('Duration : {} seconds'.format(time.time()-start))

Duration : 187062.85696220398 seconds


In [60]:
#check the best parameters
cv.best_params_

{'clf__estimator__max_depth': None,
 'clf__estimator__max_features': None,
 'clf__estimator__n_estimators': 100,
 'text_pipeline__vect__max_df': 0.85}

In [61]:
cv.best_estimator_

Pipeline(steps=[('text_pipeline',
                 Pipeline(steps=[('vect',
                                  CountVectorizer(max_df=0.85,
                                                  tokenizer=<function tokenize at 0x7f901ba7f9d0>)),
                                 ('tfidf', TfidfTransformer())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_features=None)))])

In [206]:
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [218]:
#X, y, category = load_data('DisasterResponse.db')

In [219]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [202]:
cv.score(X_train, y_train)

0.9861042994932157

In [203]:
cv.score(X_test, y_test)

0.2663699936427209

In [220]:
print(classification_report(y_test, y_pred, target_names=category, zero_division=0))

                        precision    recall  f1-score   support

               related       0.84      0.90      0.87      6012
               request       0.71      0.52      0.60      1313
                 offer       0.00      0.00      0.00        36
           aid_related       0.64      0.61      0.62      3255
          medical_help       0.37      0.25      0.30       629
      medical_products       0.44      0.29      0.35       406
     search_and_rescue       0.32      0.14      0.20       202
              security       0.14      0.03      0.04       151
              military       0.33      0.27      0.29       244
           child_alone       0.00      0.00      0.00         0
                 water       0.60      0.56      0.58       493
                  food       0.65      0.66      0.66       861
               shelter       0.54      0.47      0.50       715
              clothing       0.55      0.41      0.47       114
                 money       0.34      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#naive baise
#logistic regression
#linearscv

In [221]:
model = model_pipeline(MultinomialNB())

start = time.time()
# train pipeline
model.fit(X_train, y_train)
print('Duration : {} seconds'.format(time.time()-start))

# predict classifiers on test data
y_pred = model.predict(X_test)

Duration : 87.92979288101196 seconds


In [223]:
model.score(X_train, y_train)

0.21569482288828337

In [222]:
model.score(X_test, y_test)

0.16719643992371266

In [228]:
X, y, category = load_data('../data/DisasterResponse.db')

In [229]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [230]:
model = model_pipeline(LogisticRegression())

start = time.time()
# train pipeline
model.fit(X_train, y_train)
print('Duration : {} seconds'.format(time.time()-start))

# predict classifiers on test data
y_pred = model.predict(X_test)

Duration : 89.45406794548035 seconds


In [231]:
model.score(X_train, y_train)

0.3634877384196185

In [232]:
model.score(X_test, y_test)

0.2953591862682772

In [107]:
model_pipeline(LogisticRegression()).get_params()

{'memory': None,
 'steps': [('text_pipeline', Pipeline(steps=[('vect',
                    CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>)),
                   ('tfidf', TfidfTransformer())])),
  ('clf', MultiOutputClassifier(estimator=LogisticRegression()))],
 'verbose': False,
 'text_pipeline': Pipeline(steps=[('vect',
                  CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>)),
                 ('tfidf', TfidfTransformer())]),
 'clf': MultiOutputClassifier(estimator=LogisticRegression()),
 'text_pipeline__memory': None,
 'text_pipeline__steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>)),
  ('tfidf', TfidfTransformer())],
 'text_pipeline__verbose': False,
 'text_pipeline__vect': CountVectorizer(tokenizer=<function tokenize at 0x7f901ba7f9d0>),
 'text_pipeline__tfidf': TfidfTransformer(),
 'text_pipeline__vect__analyzer': 'word',
 'text_pipeline__vect__binary': False,
 'text_pipeline__vect__decode_error': 'st

In [123]:
import warnings
warnings.filterwarnings('ignore')

parameters = [{
    'text_pipeline__vect__max_df': [.85, 1]
   ,'clf__estimator__penalty': ['l1', 'l2']
   ,'clf__estimator__solver': ['saga', 'liblinear']
   ,'clf__estimator__C': [0.01, 0.1, 1., 10., 100.]},
    {'text_pipeline__vect__max_df': [.85, 1]
   ,'clf__estimator__penalty': ['l2']
   ,'clf__estimator__solver': ['lbfgs', 'saga', 'liblinear']
   ,'clf__estimator__C': [0.01, 0.1, 1., 10., 100.]}]

start = time.time()
logreg_cv = GridSearchCV(model_pipeline(LogisticRegression()), parameters, n_jobs=4, cv=3)
    
logreg_cv.fit(X_train, y_train)
print('Duration : {} seconds'.format(time.time()-start))

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

...

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/mila/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: 

Duration : 13459.180785894394 seconds


In [124]:
y_pred = logreg_cv.predict(X_test)

In [235]:
logreg_cv.score(X_train, y_train)

0.3367302452316076

In [234]:
logreg_cv.score(X_test, y_test)

0.3183725365543547

In [126]:
logreg_cv.best_params_

{'clf__estimator__C': 1.0,
 'clf__estimator__penalty': 'l1',
 'clf__estimator__solver': 'saga',
 'text_pipeline__vect__max_df': 0.85}

### 9. Export your model as a pickle file

In [75]:
import pickle

In [76]:
pickle.dump(cv, open('model.pkl','wb'))

In [127]:
pickle.dump(logreg_cv, open('model_logreg.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.